# Physics 641 - Ising Model Project

## Tucker Knaak - Department of Physics, Creighton University - Spring 2024

#### This code defines the Ising2D class used to investigate the physics of a two-dimensional Ising model on a square lattice.  The following functions can be called by the user to utilize this code.

#### 1.  $\_\_$init$\_\_$(L, H, T): The user inputs the lattice size, the constant external magnetic field, and the temperature of the system.  This function then sets the relevant variables of the class.

#### 2. Metropolis(N, nn,$\ ^*$H_field): The user inputs the number of steps for the algorithm, the number of nearest neighbors interacting with each spin, and a varying external magnetic field.  This function then conducts the Metropolis algorithm of flipping spins for favorable energy changes or random probabilities for the given number of steps to find the equilibrium state.

#### 3. plot_equilibrium(title, $\ ^*$colors): The user inputs the colors and the title of the plots.  This function then creates a table of the algorithm parameters and the thermodynamic variables, plots the mean energy and magnetization per particle during the algorithm, and shows the orientation of the spins on the lattice at equilibrium.

#### 4. animate_spins(): There are no inputs from the user.  This function then animates the flipping of the spins during the algorithm.

In [1]:
'''Required Libraries'''
import matplotlib.animation as animation
import matplotlib.gridspec as gridspec
import matplotlib.pyplot as plt
import numpy as np
import matplotlib

'''Required Functions'''
from scipy.odr import ODR, Model, Data, RealData
from IPython.display import HTML, display
from scipy.special import ellipk, ellipe
from matplotlib.colors import Normalize

'''Adjust Animation Size'''
matplotlib.rcParams['animation.embed_limit'] = 2**128

In [2]:
class Ising2D():
    
    '''Internal function to initialize and store the relevant variables of the class'''
    def __init__(self, lattice_size: int, H: float, T: float):
        
        '''Lattice'''
        self.L = lattice_size   #size of the lattice given by the user
        self.N = self.L**2      #number of spins on the lattice
        self.num_neighbors = 0  #number of interacting neighbors given by the user
        
        '''Spins'''
        self.spins = np.ones((self.L, self.L), dtype = float)  #array of spins oriented up
        self.spins_list = []                                   #store orientation of spins during algorithm
        
        '''Constants'''
        self.u = 1   #magnetic moment
        self.J = 1   #interaction energy
        self.H = H   #external magnetic field given by the user
        self.T = T   #temperature of the system given by the user
        self.kB = 1  #Boltzmann's constant
        self.H_field = np.array([], dtype = float)  #non-homogeneous magnetic field on lattice given by the user
        
        '''Thermodynamic Variables'''
        self.E = 0  #energy
        self.M = 0  #magnetization
        self.e = 0  #energy per particle
        self.m = 0  #magnetization per particle
        self.C = 0  #heat capacity
        self.X = 0  #magnetic susceptibility
        self.E_list = []  #store energy
        self.M_list = []  #store magnetization
        self.e_list = []  #store energy per particle
        self.m_list = []  #store magnetization per particle
        
        '''Algorithm'''
        self.num_steps = 0    #number of steps for algorithm given by the user
        self.acc_steps = 0    #number of accepted flips during the algorithm
        self.steps_list = []  #store steps during algorithm
        
        '''Plots & Animations'''
        self.title = ''         #title of plots given by the user
        self.num_frames = 100   #number of frames in the animation of the spins
        self.save_spins = None  #bool to save spins for the animation
        
        
    '''Internal function to append the relevant data in the Metropolis algorithm'''
    def _append(self, step: int):
        self.E_list.append(self.E)
        self.M_list.append(self.M)
        if self.save_spins and step % (self.num_steps // self.num_frames) == 0:
            self.spins_list.append(np.copy(self.spins))
        
        
    '''Internal function to calculate the thermodynamic varialbes describing the system at equilibrium'''
    def _thermodynamic_variables(self):
        
        '''Consider energy and magnetization after system reaches equilibrium'''
        equilibrium = 4 * self.num_steps // 5  #we ensure self.num_steps >> steps required to reach equilibrium
        
        '''Mean energy per particle'''
        E_mean = np.mean(self.E_list[equilibrium:])          #mean energy at equilibrium
        E_var = np.var(self.E_list[equilibrium:], ddof = 1)  #variance of energy at equilibrium
        self.e = E_mean / self.N                             #e = E / N
        self.e_list = [E / self.N for E in self.E_list]      #list of mean energy per particle
        
        '''Mean magnetization per particle'''
        M_mean = np.mean(self.M_list[equilibrium:])                     #mean magnetization at equilibrium
        Mabs_var = np.var(np.abs(self.M_list[equilibrium:]), ddof = 1)  #variance of abs(magnetization) at equilibrium
        self.m = M_mean / self.N                                        #m = M / N
        self.m_list = [M / self.N for M in self.M_list]                 #list of mean magnetization per particle
        
        '''Heat capacity per particle'''
        self.C = E_var / (self.kB * self.T**2 * self.N)  #C = (< E^2 > - < E >^2) / (kB * T^2 * N)
        
        '''Zero-field susceptibility per particle'''
        self.X = Mabs_var / (self.kB * self.T * self.N)  #X = (< M^2 > - < |M| >^2) / (kB * T * N)
        
        
    '''Internal function to iterate one step in the Metropolis algorithm'''
    def _step(self, step: int):
        
        '''Choose random spin to flip'''
        i, j = np.random.randint(0, self.L, size = 2)
        
        '''Sum spins of the given number of nearest neighbors of chosen spin'''
        sum_neighbors = 0
        for n in range(1, self.num_neighbors + 1):
            ln = self.spins[i, (j - n) % self.L]  #nth neighbor left
            rn = self.spins[i, (j + n) % self.L]  #nth neighbor right
            un = self.spins[(i - n) % self.L, j]  #nth neighbor up
            dn = self.spins[(i + n) % self.L, j]  #nth neighbor down
            sum_neighbors += (ln + rn + un + dn)  #update sum of nearest neighbors
        
        '''Set varying external magnetic field at chosen spin'''
        H = self.H_field[i, j] if self.H_field.size != 0 else self.H
        
        '''Calculate change in energy of flip'''
        dE = 2 * self.spins[i, j] * (self.J * sum_neighbors + self.u * H)  #dE = 2 * s_ij * (J * SUM(s_nn) + u * H)
        
        '''Check Metropolis conditions for flip'''
        if dE <= 0 or np.random.rand() < np.exp(-dE / (self.kB * self.T)):
            self.spins[i, j] *= -1          #flip spin
            self.E += dE                    #update energy
            self.M += 2 * self.spins[i, j]  #update magnetization
            self.acc_steps += 1             #update accepted steps
        
        '''Append relevant data'''
        self._append(step)
        
        
    '''Function called by the user to conduct the Metropolis algorithm for a given
       number of steps, number of neighbors, and external magnetic field'''
    def Metropolis(self, num_steps: int, num_neighbors: int, *H_field: list, save_spins = False):
        
        '''Update class variables'''
        self.num_steps = num_steps
        self.steps_list = [step for step in range(num_steps + 1)]
        self.num_neighbors = num_neighbors
        self.save_spins = save_spins
        if H_field:
            self.H_field = H_field[0]
            
        '''Set initial conditions'''
        self.E = self.N * (-2 * self.J + self.u * self.H)
        self.M = np.sum(self.spins)
        self._append(0)
        
        '''Iterate for given number of steps'''
        for step in self.steps_list[1:]:
            self._step(step)
        
        '''Calculate thermodynamic variables'''
        self._thermodynamic_variables()
        
        
    '''Function called by the user to create a table of the thermodynamic variables, plot the mean energy
       and magnetization per particle during the algorithm, and show the orientation of the spins at equilibrium'''
    def plot_equilibrium(self, *colors, title = None, save_fig = True):
        
        '''Set colors for e & m plot'''
        if colors:
            e_color, m_color = colors[0], colors[1]
        else:
            e_color, m_color = 'limegreen', 'mediumorchid'
        
        '''Set title'''
        self.title = title if title else ''
        
        '''Create figure'''
        fig = plt.figure(figsize = (16, 7))
        gs = gridspec.GridSpec(2, 4, width_ratios = [1, 1, 1, 1], height_ratios = [1, 6])
        
        '''Figure 1 --> Table of thermodynamic variables and algorithm parameters'''
        ax1 = plt.subplot(gs[0, 1:3])
        ax1.axis('off')
        if self.title:
            ax1.set_title(f'2D Ising Model (L = {self.L}, H = {self.H:.2f}, T = {self.T:.2f}) - {self.title}')
        else:
            ax1.set_title(f'2D Ising Model (L = {self.L}, H = {self.H:.2f}, T = {self.T:.2f})')
        labels = ['Steps', '% Accepted', 'Neighbors', 'E / N\u00B7J', 'm', 'C / N\u00B7kB', 'X / N\u00B7kB']
        table = [[f'{self.num_steps}', f'{self.acc_steps * 100 / self.num_steps:.2f}%', f'{self.num_neighbors * 4}',
                  f'{self.e:.4f}', f'{self.m:.4f}', f'{self.C:.4f}', f'{self.X:.4f}']]
        the_table = ax1.table(cellText = table, colLabels = labels, loc = 'center', cellLoc = 'center')
        the_table.scale(1.5, 2)
        for (row, col), cell in the_table.get_celld().items():
            if (row == 0) or (col == -1):
                cell.set_text_props(fontweight = 'bold')
            
        '''Figure 2 --> Plot of e & m during Metropolis algorithm'''
        ax2 = plt.subplot(gs[1, :2])
        ax2.set_xlabel('Steps')
        ax2.set_ylabel('Magnitude')
        ax2.set_title('Mean Energy and Magnetization Per Particle')
        ax2.grid(True, linestyle = 'dashed', color = 'darkgray', alpha = 0.25)
        ax2.plot(self.steps_list[::self.num_steps // 1000], self.e_list[::self.num_steps // 1000],
                 linestyle = 'solid', color = e_color, label = '$E\ /\ N\,J$')
        ax2.plot(self.steps_list[::self.num_steps // 1000], self.m_list[::self.num_steps // 1000],
                 linestyle = 'solid', color = m_color, label = '$m$')
        legend = ax2.legend(loc = 'best', fancybox = False)
        frame = legend.get_frame()
        frame.set_facecolor('white')
        frame.set_edgecolor('black')
        frame.set_linewidth(1)
        
        '''Figure 3 --> Heat map of orientation of spins at equilibrium'''
        ax3 = plt.subplot(gs[1, 2:])
        ax3.set_aspect('equal', 'box')
        ax3.set_xticks([])
        ax3.set_yticks([])
        ax3.set_title('Orientation of Spins at Equilibrium')
        im = ax3.imshow(self.spins, cmap = 'binary_r', interpolation = 'nearest', norm = Normalize(vmin = -1, vmax = 1))
        cbar = fig.colorbar(im, ax = ax3, ticks = [-1, 1], boundaries = [-2, 0, 2])
        cbar.ax.set_yticklabels(['Down', 'Up'])
        cbar.ax.tick_params(left = False, right = False)
        
        '''Save figure'''
        if save_fig:
            if self.title:
                fig.savefig('c:/Users/Tucker Knaak/Downloads/L{L}_H{H}_T{T}_NN{NN}_{field}_equilibrium.png'.format(
                            L = self.L, H = self.H, T = self.T, field = self.title.split()[0].lower(),
                            NN = 4 * self.num_neighbors), bbox_inches = 'tight', pad_inches = 0.5)
            else:
                fig.savefig('c:/Users/Tucker Knaak/Downloads/L{L}_H{H}_T{T}_NN{NN}_equilibrium.png'.format(
                            L = self.L, H = self.H, T = self.T, NN = 4 * self.num_neighbors),
                            bbox_inches = 'tight', pad_inches = 0.5)
            
        
    '''Internal function to update the frame of the animation of the flipping spins'''
    def _update_spins(self, frame, fig, ax):
        
        '''Clear last figure'''
        ax.clear()
        
        '''New figure'''
        step = frame * self.num_steps // self.num_frames
        ax.set_aspect('equal', 'box')
        ax.set_xticks([])
        ax.set_yticks([])
        ax.set_title(f'{self.L}x{self.L} Lattice of Spins (H = {self.H:.2f}, T = {self.T:.2f}K, Step = {step})')
        im = ax.imshow(self.spins_list[frame], cmap = 'binary_r', interpolation = 'nearest',
                       norm = Normalize(vmin = -1, vmax = 1))
        
        '''Return Artist elements'''
        return fig, ax
        
        
    '''Function called by the user to animate the flipping of the spins during the Metropolis algorithm'''
    def animate_spins(self, save_fig = True):
        
        '''Create figure'''
        fig, ax = plt.subplots(1, 1, figsize = (7, 7))
        ax.set_aspect('equal', 'box')
        ax.set_xticks([])
        ax.set_yticks([])
        
        '''Create animation'''
        ani = animation.FuncAnimation(fig, self._update_spins, interval = 75, frames = self.num_frames + 1,
                                      fargs = (fig, ax), repeat = False)
        html = HTML(ani.to_jshtml())
        display(html)
        plt.close()
        
        '''Save animation'''
        if save_fig:
            if self.title:
                f = r'c:/Users/Tucker Knaak/Downloads/L{L}_H{H}_T{T}_NN{NN}_{field}_equilibrium.gif'.format(
                      L = self.L, H = self.H, T = self.T, field = self.title.split()[0].lower(),
                      NN = 4 * self.num_neighbors)
            else:
                f = r'c:/Users/Tucker Knaak/Downloads/L{L}_H{H}_T{T}_NN{NN}_equilibrium.gif'.format(
                      L = self.L, H = self.H, T = self.T, NN = 4 * self.num_neighbors)
            writergif = animation.PillowWriter(fps = 10)
            ani.save(f, writer = writergif)